In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import emoji
import re
from typing import Any
from textblob import TextBlob
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [1]:

data = pd.read_excel('data oktober-des 2022 etilang tanpa lang dan etle.xlsx')
df = pd.DataFrame(data)
df.head()

,0
0,"Kembangkan ETLE, 7 Polda Dapat Penghargaan di ..."
1,Sosialisasi etle dan app smart city\n\n#polsek...
2,"Berita populer kumparanOTO, Selasa (13/12) det..."
3,"Mimin kasih info ya gaes, kita mau uji coba ET..."
4,personil lalu lintas polres badung melaksanaka...


In [44]:
def remove_enter(val: str)->str:
    return ' '.join(val.split())

def remove_emoji():
    emojis = sorted(emoji.EMOJI_DATA, key=len, reverse=True)
    pattern = u'(' + u'|'.join(re.escape(u) for u in emojis) + u')'
    return re.compile(pattern)

begone_emoji = remove_emoji()

def preprocessing_data(val) -> str:
    result = val.lower().strip()
    result = re.sub(r'(@|https?)\S+|#[A-Za-z0-9_]+', '', result).replace("&amp;", "dan")
    result = re.sub(r'RT[\s]+','',result)
    result = re.sub('[^.,a-zA-Z0-9 \n\.]', '', result)
    result = re.sub(r'\d', '', result)
    result = re.sub(r'\.+', " . ", result)
    result = re.sub(r'\,+', " , ", result)

    result = remove_enter(result)
    result = begone_emoji.sub(repl='', string=result)
    return result

factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()
stopword = factory.create_stop_word_remover()

def remove_stopword(val):
    removed = stopword.remove(val)
    return removed
stemmer = StemmerFactory().create_stemmer()
def stemming(val):
    hasil = stemmer.stem(val)
    return hasil

def polarity_decider(val):
    analize = TextBlob(val)
    processed = analize.translate(from_lang='id', to='en')
    if processed.polarity > 0:
        sentimen = 1
    elif processed.polarity < 0:
        sentimen = -1
    else:
        sentimen = 0
    return sentimen, processed


In [39]:
from langdetect import detect
text = df[0][0]
cleaned = preprocessing_data(text)
test = detect(cleaned)
print(test)
print(cleaned)


id
kembangkan etle , polda dapat penghargaan di rakernis fungsi gakkum
